In [31]:
def oversample(input_file):
    img_container = [[] for i in range(20)]
    oversample_data = []

    for line in input_file:
        line = line.strip()
        oversample_data.append(line)
        
        img_labels = line.split(" ")[5::5]
        img_labels_counter = np.zeros(20)  
        for i in img_labels:
            img_labels_counter[int(i)] += 1
        
        # to deal with the problem of large amount of human
        if img_labels_counter[14] > 0:
            img_container[14].append(line)
        else:
            img_container[np.argmax(img_labels_counter)].append(line)


    imgNum_of_class = [len(img) for img in img_container]
    threshold = max(imgNum_of_class) // 6
    
    sampled_data = []
    for i in range(20):
        if imgNum_of_class[i] < threshold :
            sampled_data.extend(random.choices(img_container[i], k = threshold - imgNum_of_class[i]))

    oversample_data.extend(sampled_data)
    
    with open('oversample_training_data.txt', 'w') as f:
        for item in oversample_data:
            f.write("%s\n" % item)
            
##################################################

    oversample_file = open('oversample_training_data.txt', 'r')
    
    img_container = [[] for i in range(20)]
    
    for line in oversample_file:
        line = line.strip()
        
        img_labels = line.split(" ")[5::5]
        img_labels_counter = np.zeros(20)  
        for i in img_labels:
            img_labels_counter[int(i)] += 1
        
        img_container[np.argmax(img_labels_counter)].append(line)

    imgNum_of_class = [len(img) for img in img_container]
    
    aug_probability = (np.max(imgNum_of_class) - imgNum_of_class) / imgNum_of_class / 10
    return aug_probability

In [33]:
aug('./pascal_voc_training_data.txt', [])

In [37]:
#YOLO = yolo(training=True)
#YOLO.build(input_shape=(None, IMAGE_SIZE, IMAGE_SIZE, 3))

In [39]:
img_inputs = keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
print(img_inputs)

base_model = keras.applications.InceptionResNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, weights='imagenet')
# # base_model = keras.applications.ResNet152(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, weights='imagenet')
base_model.trainable = False
x = base_model(img_inputs,  training=False)
x = base_model(img_inputs)
x = conv_leaky_relu(x, 1024, 3, 1)
x = conv_leaky_relu(x, 1024, 3, 2)
x = conv_leaky_relu(x, 1024, 3, 1)
x = conv_leaky_relu(x, 1024, 3, 1)
x = layers.Flatten()(x)
x = layers.Dense(4096, kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.01))(x)
x = layers.LeakyReLU(0.1)(x)
outputs = layers.Dense(1470, kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.01))(x)

YOLO = keras.Model(inputs=img_inputs, outputs=outputs, name="YOLO")

In [40]:
YOLO.summary()

Model: "yolo_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, None, None, 1536)  54336736  
_________________________________________________________________
conv_leaky_relu_4 (conv_leak multiple                  14159872  
_________________________________________________________________
conv_leaky_relu_5 (conv_leak multiple                  9441280   
_________________________________________________________________
conv_leaky_relu_6 (conv_leak multiple                  9441280   
_________________________________________________________________
conv_leaky_relu_7 (conv_leak multiple                  9441280   
_________________________________________________________________
flatten_1 (Flatten)          multiple                  0         
_________________________________________________________________
dense_3 (Dense)              multiple                  15099

In [41]:
# base boxes (for loss calculation)
base_boxes = np.zeros([CELL_SIZE, CELL_SIZE, 4])

# initializtion for each cell
for y in range(CELL_SIZE):
    for x in range(CELL_SIZE):
        base_boxes[y, x, :] = [IMAGE_SIZE / CELL_SIZE * x, 
                               IMAGE_SIZE / CELL_SIZE * y, 0, 0]

base_boxes = np.resize(base_boxes, [CELL_SIZE, CELL_SIZE, 1, 4])
base_boxes = np.tile(base_boxes, [1, 1, BOXES_PER_CELL, 1])

In [44]:
dataset = DatasetGenerator().generate()

optimizer = tf.keras.optimizers.Adam(LEARNING_RATE)

train_loss_metric = tf.keras.metrics.Mean(name='loss')

ckpt = tf.train.Checkpoint(epoch=tf.Variable(0), net=YOLO)

manager = tf.train.CheckpointManager(ckpt, './ckpts/YOLO', max_to_keep=10, checkpoint_name='yolo')

In [45]:
@tf.function
def train_step(image, labels, objects_num):
    with tf.GradientTape() as tape:
        outputs = YOLO(image)
        class_end = CELL_SIZE * CELL_SIZE * NUM_CLASSES
        conf_end = class_end + CELL_SIZE * CELL_SIZE * BOXES_PER_CELL
        class_probs = tf.reshape(outputs[:, 0:class_end], (-1, 7, 7, 20))
        confs = tf.reshape(outputs[:, class_end:conf_end], (-1, 7, 7, 2))
        boxes = tf.reshape(outputs[:, conf_end:], (-1, 7, 7, 2*4))
        predicts = tf.concat([class_probs, confs, boxes], 3)

        loss = yolo_loss(predicts, labels, objects_num)
        train_loss_metric(loss)

    grads = tape.gradient(loss, YOLO.trainable_weights)
    optimizer.apply_gradients(zip(grads, YOLO.trainable_weights))

In [46]:
print("{}, start training.".format(datetime.now()))
for i in range(EPOCHS):
    train_loss_metric.reset_states()
    ckpt.epoch.assign_add(1)

    for idx, (image, labels, objects_num) in enumerate(dataset):
        train_step(image, labels, objects_num)

    print("{}, Epoch {}: loss {:.2f}".format(datetime.now(), i+1, train_loss_metric.result()))

    save_path = manager.save()
    print("Saved checkpoint for epoch {}: {}".format(int(ckpt.epoch), save_path))    

2020-12-03 18:26:33.387757, start training.
2020-12-03 18:34:22.157202, Epoch 1: loss 11.49
Saved checkpoint for epoch 1: ./ckpts/YOLO/yolo-1
2020-12-03 18:42:04.518460, Epoch 2: loss 8.07
Saved checkpoint for epoch 2: ./ckpts/YOLO/yolo-2
2020-12-03 18:49:47.588797, Epoch 3: loss 7.14
Saved checkpoint for epoch 3: ./ckpts/YOLO/yolo-3
2020-12-03 18:57:30.258279, Epoch 4: loss 6.58
Saved checkpoint for epoch 4: ./ckpts/YOLO/yolo-4
2020-12-03 19:05:13.122952, Epoch 5: loss 6.12
Saved checkpoint for epoch 5: ./ckpts/YOLO/yolo-5
2020-12-03 19:12:55.839453, Epoch 6: loss 5.65
Saved checkpoint for epoch 6: ./ckpts/YOLO/yolo-6
2020-12-03 19:20:39.059359, Epoch 7: loss 5.26
Saved checkpoint for epoch 7: ./ckpts/YOLO/yolo-7
2020-12-03 19:28:22.140769, Epoch 8: loss 4.94
Saved checkpoint for epoch 8: ./ckpts/YOLO/yolo-8
2020-12-03 19:36:05.067971, Epoch 9: loss 4.57
Saved checkpoint for epoch 9: ./ckpts/YOLO/yolo-9
2020-12-03 19:43:48.648398, Epoch 10: loss 4.32
Saved checkpoint for epoch 10: ./c

KeyboardInterrupt: 

In [47]:
from ensemble_boxes import *
def process_outputs11111(outputs):
    """
    Process YOLO outputs into bou
    """

    class_end = CELL_SIZE * CELL_SIZE * NUM_CLASSES
    conf_end = class_end + CELL_SIZE * CELL_SIZE * BOXES_PER_CELL
    class_probs = np.reshape(outputs[:, 0:class_end], (-1, 7, 7, 20))
    confs = np.reshape(outputs[:, class_end:conf_end], (-1, 7, 7, 2))
    boxes = np.reshape(outputs[:, conf_end:], (-1, 7, 7, 2*4))
    predicts = np.concatenate([class_probs, confs, boxes], 3)

    p_classes = predicts[0, :, :, 0:20]
    C = predicts[0, :, :, 20:22]
    coordinate = predicts[0, :, :, 22:]

    p_classes = np.reshape(p_classes, (CELL_SIZE, CELL_SIZE, 1, 20))
    C = np.reshape(C, (CELL_SIZE, CELL_SIZE, BOXES_PER_CELL, 1))
    
    P = C * p_classes
    #P's shape [7, 7, 2, 20]

    #choose the bboxes that have confidence non less than 0.5  (sample at most 5 boxes)
    threshold = 0.025
    sample = 20
    _P = P.flatten()
    indexes = np.asarray([x for x in np.argsort(_P)[-sample:] if _P[x] >= threshold])
#     print(indexes)
    # if no box is included(indexes == []), get the most confident one (temporary solution)
    if len(indexes) == 0:
        indexes = np.asarray([np.argmax(_P)])
    idx_cnts = len(indexes)
    indexes = np.unravel_index(indexes, P.shape)
    max_conf = P[indexes]
    xmin = []
    ymin = []
    xmax = []
    ymax = []
    class_num = []
    
    indexes = np.asarray(indexes)
    for cnts in range(idx_cnts):
        index = indexes[:,cnts]
    
#     max_conf = np.max(P)
#     index = np.argmax(P)

#     index = np.unravel_index(index, P.shape)

        _class_num = index[3]

        coordinate = np.reshape(coordinate, 
                                (CELL_SIZE, 
                                 CELL_SIZE,
                                 BOXES_PER_CELL, 
                                 4))

        max_coordinate = coordinate[index[0], index[1], index[2], :]

        xcenter = max_coordinate[0]
        ycenter = max_coordinate[1]
        w = max_coordinate[2]
        h = max_coordinate[3]

        xcenter = (index[1] + xcenter) * (IMAGE_SIZE/float(CELL_SIZE))
        ycenter = (index[0] + ycenter) * (IMAGE_SIZE/float(CELL_SIZE))

        w = w * IMAGE_SIZE
        h = h * IMAGE_SIZE

        _xmin = xcenter - w/2.0
        _ymin = ycenter - h/2.0

        _xmax = _xmin + w
        _ymax = _ymin + h
        
        if(_xmin <= 0):
            _xmin = 0.01
        if(_ymin <= 0):
            _ymin = 0.01
        if(_xmax >= IMAGE_SIZE):
            _xmax = IMAGE_SIZE-0.01
        if(_ymax >= IMAGE_SIZE):
            _ymax = IMAGE_SIZE-0.01
            
        xmin.append(_xmin)
        ymin.append(_ymin)
        xmax.append(_xmax)
        ymax.append(_ymax)
        class_num.append(_class_num)
    
    boxes_list = []
    scores_list = max_conf
    labels_list = class_num


    weights =[3]
    iou_thr = 0.5
    skip_box_thr = 0.0001
    sigma = 0.1

    xmin      = np.asarray(xmin)
    ymin      = np.asarray(ymin) 
    xmax      = np.asarray(xmax)
    ymax      = np.asarray(ymax)
    class_num = np.asarray(class_num)
    max_conf  = np.asarray(max_conf)

    for i in range(len(xmin)):
        box_temp = [ xmin[i]/448., ymin[i]/448., xmax[i]/448., ymax[i]/448.]
        boxes_list.append(box_temp)

    boxes, scores, labels = nms([boxes_list], [scores_list], [labels_list], weights=None, iou_thr=iou_thr)

    xmin_NMS = []
    ymin_NMS = []
    xmax_NMS = []
    ymax_NMS = []
    for i in range(len(labels)):
#             class_name = classes_name[labels[i]]
        xmin_NMS.append(int(boxes[i][0]*448))
        ymin_NMS.append(int(boxes[i][1]*448))
        xmax_NMS.append(int(boxes[i][2]*448))
        ymax_NMS.append(int(boxes[i][3]*448))   

    xmin_NMS_big = []
    ymin_NMS_big = []
    xmax_NMS_big = []
    ymax_NMS_big = []
    label_NMS_big= []
    scores_NMS_big=[]
    
    for i in range(len(labels)):
        if((xmax_NMS[i]-xmin_NMS[i] >= 10) and (ymax_NMS[i]-ymin_NMS[i] >= 10)):
                xmin_NMS_big.append(xmin_NMS[i])
                ymin_NMS_big.append(ymin_NMS[i])
                xmax_NMS_big.append(xmax_NMS[i])
                ymax_NMS_big.append(ymax_NMS[i])
                label_NMS_big.append(labels[i])
                scores_NMS_big.append(scores[i])
    return np.asarray(xmin_NMS_big), np.asarray(ymin_NMS_big), np.asarray(xmax_NMS_big), np.asarray(ymax_NMS_big), np.asarray(label_NMS_big), np.asarray(scores_NMS_big)
#     return np.asarray(xmin_NMS), np.asarray(ymin_NMS), np.asarray(xmax_NMS), np.asarray(ymax_NMS), np.asarray(labels), np.asarray(scores)